<a href="https://colab.research.google.com/github/Arun-nexus/deep_learning/blob/main/spam_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Arun\Downloads\spam (1).csv",encoding="latin-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
from nltk import word_tokenize,WordNetLemmatizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
x=df["v2"]
y=df["v1"]

In [ ]:
x.head()
x=x.tolist()
word=[]
for i in range(0,3):
    print(x[i])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


In [ ]:
def preprocess(words):
    lemmatizer=WordNetLemmatizer()
    tokens=word_tokenize(words)
    lemmatized=[lemmatizer.lemmatize(i) for i in tokens]
    return tokens

dict=[preprocess(j) for j in x]

In [ ]:
from collections import Counter
import torch.nn as nn

In [ ]:
joined_data=[" ".join(row) for row in dict]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from torch.utils.data import TensorDataset,DataLoader
import math

In [ ]:
print(repr(joined_data[0]))
print(repr(joined_data[1]))
print(repr(joined_data[2]))

'Go until jurong point , crazy .. Available only in bugis n great world la e buffet ... Cine there got amore wat ...'
'Ok lar ... Joking wif u oni ...'
"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005 . Text FA to 87121 to receive entry question ( std txt rate ) T & C 's apply 08452810075over18 's"


In [ ]:

vectorizer=TfidfVectorizer()
vectorized_data=vectorizer.fit_transform(joined_data)

In [ ]:
vectorized_data=vectorized_data.todense()


In [ ]:
print(vectorized_data[0])
print((vectorized_data[0]!=0).sum())
print(vectorized_data.shape)

tensor([0, 0, 0,  ..., 0, 0, 0])
tensor(0)
torch.Size([5572, 8660])


In [ ]:
vocab_size=len(vectorizer.get_feature_names_out())
vectorized_data=torch.tensor(vectorized_data,dtype=torch.long)

C:\Users\Arun\AppData\Local\Temp\ipykernel_12864\414006721.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vectorized_data=torch.tensor(vectorized_data,dtype=torch.long)


In [ ]:
x=vectorized_data

In [ ]:
y=y.map({"spam":1,"ham":0})
y=torch.tensor(y,dtype=torch.long)

In [ ]:
print(x.shape)
print(y.shape)
y

torch.Size([5572, 8660])
torch.Size([5572])


tensor([0, 0, 1,  ..., 0, 0, 0])

In [ ]:
dataset=TensorDataset(x,y)

In [ ]:
from sklearn.model_selection import train_test_split
training_data,validation_data=train_test_split(dataset,test_size=0.2,random_state=42)
train_data_loader=DataLoader(training_data,batch_size=512,shuffle=True)
val_data_loader=DataLoader(validation_data,batch_size=512)

In [ ]:
class positional_encoding(nn.Module):
    def __init__(self,d_model,max_len=1000):
        super().__init__()
        pe=torch.zeros(max_len,d_model)
        position=torch.arange(0,max_len).unsqueeze(1)
        div_term=torch.exp(torch.arange(0,d_model,2)*-math.log(10000.0)/d_model)
        pe[:,0::2]=torch.sin(position*div_term)
        pe[:,1::2]=torch.cos(position*div_term)
        pe=pe.unsqueeze(0)
        self.register_buffer("pe",pe)

    def forward(self,x):
        return x+self.pe[:,:x.size(1)]

In [ ]:
class multihead_attention(nn.Module):
    def __init__(self,d_model,num_heads=8):
        super().__init__()
        assert d_model % num_heads==0

        self.d_k = d_model//num_heads
        self.num_heads=num_heads
        self.k_lin=nn.Linear(d_model,d_model)
        self.q_lin=nn.Linear(d_model,d_model)
        self.v_lin=nn.Linear(d_model,d_model)
        self.out=nn.Linear(d_model,d_model)

    def forward(self,q,k,v,mask=None):
        batch_size=q.size(0)

        def transform(x,linear):
            x=linear(x)
            x=x.view(batch_size,-1,self.num_heads,self.d_k)
            return x.transpose(1,2)
        q,k,v=transform(q,self.q_lin),transform(k,self.k_lin),transform(v,self.v_lin)

        scores = torch.matmul(q,k.transpose(-2,-1)/math.sqrt(self.d_k))
        if mask is not None:
            scores = scores.masked_fill(mask==0,-1e9)
        attn = torch.softmax(scores,dim=-1)
        output = torch.matmul(attn,v)
        output=output.transpose(1,2).contiguous().view(batch_size,-1,self.d_k*self.num_heads)
        return self.out(output)

In [ ]:
class feedforward(nn.Module):
    def __init__(self,d_model,d_ff=2048,dropout=0.2):
        super().__init__()
        self.feature=nn.Sequential(
        nn.Linear(d_model,d_ff),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(d_ff,d_model)
        )
    def forward(self,x):
        return self.feature(x)


In [ ]:
class encoder(nn.Module):
    def __init__(self,d_model,num_heads,d_ff,dropout=0.2):
        super().__init__()
        self.attn=multihead_attention(d_model,num_heads)
        self.ff=feedforward(d_model,d_ff,dropout)
        self.norm1=nn.LayerNorm(d_model)
        self.norm2=nn.LayerNorm(d_model)
        self.drop=nn.Dropout(dropout)
    def forward(self,x,mask=None):
        x2=self.norm1(x+self.drop(self.attn(x,x,x,mask)))
        x=self.norm2(x2+self.drop(self.attn(self.ff(x2))))
        return x

In [ ]:
class encoderlayer(nn.Module):
    def __init__(self,vocab_size,d_model,num_layers,num_heads,d_ff,max_len=512):
        super().__init__()
        self.embedding=nn.Embedding(vocab_size,d_model)
        self.pe=positional_encoding(d_model,max_len)
        self.layers=nn.ModuleList([encoder(d_model,num_heads,d_ff) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self,src,mask=None):
        x=self.embedding(src)
        x=self.pe(x)
        for layer in self.layers:
            x=layer(x,mask)
        return self.norm(x)

In [ ]:
from torch.amp import GradScaler,autocast
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
device=("cuda" if torch.cuda.is_available() else "cpu")
model = encoderlayer(vocab_size=vocab_size,d_model=512,num_layers=6,num_heads=8,d_ff=2048).to(device)
model_loss=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer,mode="min",factor=0.5,patience=2)

In [ ]:
class early_stopping():
    def __init__(self,min_delta,patience):
        self.min_delta=min_delta
        self.patience=patience
        self.counter=0
        self.best_loss=float("inf")
        self.early_stop=False

    def __call__(self,val_loss):
        if self.min_delta>val_loss-self.best_loss:
            counter =0
            self.best_loss=val_loss
        else:
            self.counter+=1
            if self.counter>=self.patience:
                self.early_stop=True

In [ ]:
epochs=50
training_loss=[]
training_acc=[]
stopper=early_stopping(min_delta=1e-4,patience=4)
scaler=GradScaler()

for epoch in range(epochs):
    runn_loss,correct,total=0,0,0
    for x,y in val_data_loader:
        x,y=x.to(device),y.to(device)
        optimizer.zero_grad()
        with autocast("cuda"):
            output=model(x)
            loss=model_loss(output,y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        runn_loss+=loss.item()
        _,pred=torch.max(output,1)
        correct+=(pred==y).sum().item()
        total+=y.size(0)
    running_loss=runn_loss/len(y)
    acc=100*correct/total
    training_loss.append(running_loss)
    training_acc.append(acc)

    print(f"epoch: {epoch+1} training_loss: {running_loss:.4f} training acc {acc:.2f}")

    stopper(running_loss)
    if stopper.early_stop:
        print("early_stopping triggered")
        break